# OWL-Demo1 Run OWL (all nodes) and download pcap files

In [ ]:
from IPython.core.getipython import get_ipython

# Import libraries
%run ../../../setup/include/include_libraries.py
# Import selected slice name
%run ../../../slice_info/selected_slice.py


# Import topology_variables

# Use the button in the GUI automatically does it for you
if os.getenv('SELECTED_SLICE') is not None:
    SELECTED_SLICE = os.getenv('SELECTED_SLICE')

# If you manually run the notebook, please specify the slice name in the line below
#SELECTED_SLICE = 'MySlice2'

path = f'../../../slice_info/{SELECTED_SLICE}/topology_variables.ipynb'
get_ipython().run_line_magic('run', path)


## Get slice and nodes

In [ ]:
try:
    slice = fablib.get_slice(name=selected_slice)
except Exception as e:
    print(f"Fail: {e}")
#print(slice)
nodes = slice.get_nodes()

## Check whether the topology of the selected slice is eligible to run OWL
#### (1) number of nodes >2
#### (2) nodes are on ptp capable sites

In [ ]:
eligible = True

# number of nodes 
if (len(nodes)<2):
    print ('You need at lease two nodes to run OWL experiment')
    eligible = False

# nodes are on ptp capable sites 
ptp_sites = fablib.list_sites(
    output="list",
    quiet=True,
    filter_function = lambda x:x['ptp_capable'] is True and x['state'] == 'Active',
)
ptp_site_names = [x['name'] for x in ptp_sites]

#print(f"PTP-enabled and active sites: {len(ptp_site_names)} ({ptp_site_names})")
for node in nodes:
    #print (node.get_ip_routes())
    if node.get_name() != 'meas-node':
        if node.get_site() not in ptp_site_names:
            print (f'Site {node.get_site()} is not ptp capable. Stop running the notebook.')
            eligible = False



## Enter the docker image name

Image will be pulled from Docker Hub by default.
If needing bo build from a Dockerfile, use [this](./OWL-DEV_build_image_from_Dockerfile.ipynb) in the same directory and change the image name. (e.g., "owl-test:latest")

In [ ]:
# if specific tag needed, add it like: "fabrictestbed/owl:v0.1.2"
image_name="fabrictestbed/owl:0.2.0"  

# If built from Dockerfile for testing 
#image_name="owl-test:latest"  

#### (Optional) Pull Docker image on each exp node

Docker start (in the cells below) should pull the image automatically.
But if pulling the image is necessary, use these lines.

In [ ]:
# for node in nodes:
#     print(node.get_name())
#     node.execute(f"sudo docker pull {image_name}") 

## Use start_owl_all

In [ ]:
import time
if eligible is True:
    print ('Starting owl...')
    owl.start_owl_all(slice, img_name=image_name, probe_freq=10, duration=120, delete_previous=True)

## (Optional) If the method above doesn't work, try the following

### Run Packet Captures

#### Start Capturer containers

In [ ]:
# for node in nodes:
#     owl.start_owl_capturer(slice, dst_node=node, img_name=image_name, duration=30)

### Start Sender containers

In [ ]:
# for src_node in nodes:
#     for dst_node in nodes:
#         if dst_node.get_name() != src_node.get_name():
#             print(f"{src_node.get_name()} --> {dst_node.get_name()}")
#             owl.start_owl_sender(slice, src_node=src_node, dst_node=dst_node, img_name=image_name, duration=86400)

## Check the status of containers

In [ ]:
if eligible is True:
    print ('Checking owl...')
    owl.check_owl_all(slice)
    print ('Waiting 120 seconds for owl to collect data...')
    time.sleep(120)

## Check if pcap files are being generated

In [ ]:
#if elgible:
#    for node in nodes:
#        node.execute("ls -lh ~/owl-output/")

## Stop all of them

In [ ]:
if eligible is True:
    print ('Stoping owl...')
    owl.stop_owl_all(slice)

# (Optional) Download pcap files

In [ ]:
if eligible is True: 
    for node in nodes:
        if node.get_name() != 'meas-node':
            pcap_name, _ = node.execute("ls ~/owl-output")
            node.download_file(f"./{pcap_name.strip()}", f"/home/rocky/owl-output/{pcap_name.strip()}",)